In [ ]:
from importlib import reload
import IntersectionDataset
reload(IntersectionDataset)
from IntersectionDataset import IntersectionDataset2

from torch.utils.data import DataLoader
import multiprocessing
from torchvision.transforms import ToTensor
import torch
import glob
import os
import cv2
import json
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
intersections = glob.glob('dataset/test/intersection_*')
intersections

In [ ]:
def map_exit_to_class(exit_x, exit_y):
    if exit_y == 0:
        return 1  # left
    elif exit_y == 399:
        return 2  # right
    elif exit_x == 0:
        return 3  # ahead
    else:
        raise ValueError("Unexpected exit position")


In [ ]:
for i, intersection in enumerate(intersections):
    satellite = os.path.join(intersection, 'satellite.png')
    paths = glob.glob(f"{intersection}/paths/*")
    paths_png = [os.path.join(paths[i], f"path_line.png") for i in range(len(paths))]
    paths_img = [cv2.imread(paths_png[i], cv2.IMREAD_GRAYSCALE) for i in range(len(paths_png))]
    ee_dirs = [os.path.join(paths[i], f"path_line_ee.json") for i in range(len(paths))]
    ee_data = []
    for ee_dir in ee_dirs:
        with open(ee_dir) as f:
            ee_data.append(json.load(f))

    
    H, W = paths_img[0].shape

    combined = torch.full((1, H, W), 0, dtype=torch.long)

    for j, p in enumerate(paths_img):
        ee = ee_data[j]
        exit_x = ee["exit"]["x"]
        exit_y = ee["exit"]["y"]
        
        class_label = map_exit_to_class(exit_x, exit_y)
        
        p = cv2.GaussianBlur(p, (5, 5), 0)
        
        mask = torch.from_numpy(p > 0)
        
        class_label = torch.full((1, H, W), class_label, dtype=torch.long)
        
        combined += mask * class_label
    combined = combined.clamp(0, 4)

    combined_img = combined.squeeze().numpy()
    
    np.save(f"{intersection}/class_labels.npy", combined_img)
    print(f"Saved {intersection}/class_labels.npy")

In [ ]:
img = np.load(f"dataset/test/intersection_032/class_labels.npy")
plt.imshow(img, cmap="inferno")
plt.colorbar()
plt.axis('off')

In [ ]:
# find percentage of each class
for i, intersection in enumerate(intersections):
    img = np.load(f"{intersection}/class_labels.npy")
    unique, counts = np.unique(img, return_counts=True)
    percentages = np.round((counts / counts.sum()) * 100, 2)
    class_counts = {int(k): {'count': int(v), 'percentage': float(p)} for k, v, p in zip(unique, counts, percentages)}
    print(f"Intersection {i}: {class_counts}")